# Grants.gov XML extract, filtered and pushed to MongoDB
 By. Angie Marchany-Rivera & Sara Smithers  
 August 13<sup>th</sup>, 2022  
 Updated September 13<sup>th</sup>, 2022

The goal of the this project to provide a tool that allows users to search for environmental remediation grants funded by the Bipartisan Infrastructure Law. This project was completed as part of the Opportunity Project led by the US Census Bureau, The White House OSTP, and the US Department of Commerce. The following code extracts, cleans and filters available grant data from the Grants.gov website. The intial data can then be uploaded MongoDB, with additional code that can be run daily to check for new grants and updates to grants.

### Extracting the Data from the Grants.gov XML extract

In [ ]:
#pip install xmltodict
#pip install pymongo
#pip install pymongo[srv]

In [1]:
# https://openbase.com/python/xmltodict

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import xmltodict
import os
import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import sys
import pandas as pd
import numpy as np
import shutil

In [2]:
print (sys.version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


The Grants.gov database of grants is exported to an XML file everyday. The XML file is compressed into a zip file that is named 'GrantsDBExtractYYYYMMDD.zip' with the updated date of the extract. The following URL format can be used to download the data. More information about the XML extract can be found here: https://www.grants.gov/help/html/help/index.htm?rhcsh=1&callingApp=custom#t=XMLExtract%2FXMLExtract.htm

To extract the data from the XML file, first use Timestamp function to insert today's date into the the zip file URL. You will need to make a temporary directory where the zip file can be saved and opened so that the XML file can be read. From there, the XML data will need to be parsed into a dictionary. The grant data can then be extracted as a list of dictionaries, where each dictionary contains the data for one grant. The list of dictionaries can be converted to a Pandas dataframe to be cleaned. Our cleaning steps included updating object datatypes to numeric and datetime types. We aso found errors in ArchiveDate, where the years were listed as 7009 and 3010. We assumed this was due to human error and corrected the years to 2009 and 2010.

In [3]:
# Direct link to updated XML extract. 
# Example link --> https://www.grants.gov/extract/GrantsDBExtract20220813v2.zip

today = pd.Timestamp('today')
link = "https://www.grants.gov/extract/GrantsDBExtract{:%Y%m%d}v2.zip".format(today)

In [4]:
# Extract most up-to-date XML from grants.gov zip file.
# SLOW STEP - It collects the new XML file in the temp folder.

# Make a temporary directory inside local host working directory.
os.makedirs('tempDir', exist_ok = True)

# Open the link to the grants.gov xml extract zip file.
url = urlopen(link)

# Read the zipfile.
zipfile = ZipFile(BytesIO(url.read()))

# Save zipfile to temporary directory.
zipfile.extractall('tempDir/temp')

# Save the file name as a string.
filename = os.listdir('tempDir/temp')

# Read the xml file and save it to xml_data.
with open(os.path.join('tempDir/temp', filename[0]), 'r', encoding = 'utf-8') as f:
    xml_data = f.read()

# Delete the temporary directory from local host working directory.
shutil.rmtree(f'{os.getcwd()}\\tempDir')

In [5]:
# Parse the xml data into a dictionary dtype.

xml_dict = xmltodict.parse(xml_data, encoding = 'utf-8')

In [6]:
# Extract the data from the dictionary. It comes as a list of dictionaries where each dictionary
# stores the data of one grant on grants.gov.

data_list_dict = xml_dict['Grants']['OpportunitySynopsisDetail_1_0']

In [7]:
# Convert the list of dictionaries into a pandas dataframe to perform cleaning.

xml_df = pd.DataFrame(data_list_dict)

In [8]:
# Update data types. Numbers = "pd.to_numeric" & dates = "pd.to_datetime":

xml_df['OpportunityID'] = pd.to_numeric(xml_df['OpportunityID'], errors='coerce')
xml_df['AwardCeiling'] = pd.to_numeric(xml_df['AwardCeiling'], errors='coerce')
xml_df['AwardFloor'] = pd.to_numeric(xml_df['AwardFloor'], errors='coerce')
xml_df['EstimatedTotalProgramFunding'] = pd.to_numeric(xml_df['EstimatedTotalProgramFunding'], errors='coerce')
xml_df['ExpectedNumberOfAwards'] = pd.to_numeric(xml_df['ExpectedNumberOfAwards'], errors='coerce')
xml_df['PostDate'] = pd.to_datetime(xml_df['PostDate'], format='%m%d%Y', errors='ignore')
xml_df['CloseDate'] = pd.to_datetime(xml_df['CloseDate'], format='%m%d%Y', errors='ignore')
xml_df['LastUpdatedDate'] = pd.to_datetime(xml_df['LastUpdatedDate'], format='%m%d%Y', errors='ignore')


In [9]:
# Found errors on Archive Date - possible cause: human-entered.

xml_df.loc[xml_df['ArchiveDate']=='08157009','ArchiveDate'] = '08152009'
xml_df.loc[xml_df['ArchiveDate']=='03023010','ArchiveDate'] = '03022010'

In [10]:
# Update data type for Archive Date after correcting typos:

xml_df['ArchiveDate'] = pd.to_datetime(xml_df['ArchiveDate'], format='%m%d%Y')

In [ ]:
# RUN this code if you want to download the xml_df into a csv file.

# Replace pd.NaT on datetime columns to np.nan for Tableau.

# xml_df['PostDate'].replace({pd.NaT: np.nan}, inplace = True)
# xml_df['CloseDate'].replace({pd.NaT: np.nan}, inplace = True)
# xml_df['LastUpdatedDate'].replace({pd.NaT: np.nan}, inplace = True)
# xml_df['ArchiveDate'].replace({pd.NaT: np.nan}, inplace = True)

# Create CSV for exploratory analysis on Tableau Desktop.

# xml_df[xml_df['ArchiveDate'].dt.year >= 2022].to_csv('grants_active.csv', index = False)

### Filtering the data to extract grants and cooperative agreements for environmental remediation in the USA

Now that the grants.gov extract is in a clean dataframe, it can be filtered to BIL Programs. The dataframe can be first filtered by the ArchiveDate to remove any grants/cooperative agreements that have been archived prior to the current year.

There are 8 environmental remediation programs to be funded by the Bipartisan Infrastructure Law (BIL). Three agencies are responsible for funding these programs, the Department of the Interior, the Environmental Protection Agency (EPA), and the Department of Energy (DOE). 

The CFDA number is used to determine the parent_agency_number. The CFDA Number is a unique number assigned to grants/cooperative agreements (XX.XXX). The first number (XX) is the parent agency number, and the second number (XXX) is the program number. A parent_agency_number was extracted from the first part of the CFDA Number (XX) and added to the filtered dataframe. We were then able to filter by the 3 parent agencies that fund the BIL programs.

We created a new dataframe from a dictionary of these parent agencies, which we joined to the filtered dataframe on the parent_agency_number. From there added a new column called 'BIL_ERG_funds', identifying whether the grant is funded by one of the BIL Enviornmental Remediation Programs.

In [11]:
# Filter by 'ArchiveDate' >= current date 
# This will reduce the number of rows since all archived grants will be ignored.

current_date = pd.Timestamp('today')
xml_new_df = xml_df[xml_df['ArchiveDate'] >= current_date]

In [12]:
# Add new column with Parent Agency number extracted from CFDA number

xml_new_df['parent_agency_number'] = xml_new_df['CFDANumbers'].astype(str).str.split('.').str[0]

C:\Users\smith\AppData\Local\Temp\ipykernel_12532\3405912243.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xml_new_df['parent_agency_number'] = xml_new_df['CFDANumbers'].astype(str).str.split('.').str[0]


In [13]:
# Filter data to grants only funded by the 3 agencies that fund BIL programs

BIL_agencies = ['15', '66', '81']

BIL_df = xml_new_df[xml_new_df['parent_agency_number'].isin(BIL_agencies)]

In [14]:
# Create the dictionary, `ENV_Agencies_dict`, using the fromkeys() method.
col_headers = ('parent_agency_number','parent_agency_name')
BIL_Agencies_dict = dict.fromkeys(col_headers)

# Fill in the parent_agency_number and parent_agency_name
BIL_Agencies_dict['parent_agency_number'] = ['15','66','81']
BIL_Agencies_dict['parent_agency_name'] = ['Department of Interior','Environmental Protection Agency','Department of Energy']

#Create a pandas dataframe with the dictionary to be able to merge it with the ENV_df.
BIL_Agencies_df = pd.DataFrame.from_dict(BIL_Agencies_dict, dtype = object)

In [15]:
# join dataframe with parent agency name csv to include the parent agency name

filtered_df = pd.merge(BIL_df, BIL_Agencies_df, on = 'parent_agency_number')

In [16]:
# CFDA Numbers for BIL ER Programs from Build.gov with links to Sam.gov: 
# Abandoned Mine Reclamation Fund- 15.252
# Brownfields Projects- 66.818
# Brownfields State & Tribal Response Porgrams- 66.817
# Clean Energy Demonstrations on Current and Former Mine Land & Funding to Support Orphan Well Plugging- 81.089 
# Superfund- 66.802 
# Orphaned Well Site Plugging, Remediation, and Restoration- 15.018
# Direct Federal Spending for Revegetation of Mined Lands- No link to sam.gov, no CFDA number listed
# New columns include 'BIL_ERG_funds', identifying whether the grant is funded by one of the 7 BIL ER Grants.

BILPrograms = ['15.252', '66.818', '66.817', '81.089', '66.802', '15.018']

filtered_df['BIL_ERG_funds'] = np.where(filtered_df['CFDANumbers'].isin(BILPrograms), 1, 0)

In [17]:
# There seemed to be an issue with parsing the xml dict that resulted in unwanted characters 
# We were not able to correct the issue by specifying the encoding as utf-8, so we manually corrected the errors that we found

filtered_df.replace('&#8220;', '"', regex = True, inplace = True)
filtered_df.replace('&#8221;', '"', regex = True, inplace = True)
filtered_df.replace('&quot;', '"', regex = True, inplace = True)
filtered_df.replace('&#8217;', "'", regex = True, inplace = True)
filtered_df.replace('&apos;', "'", regex = True, inplace = True)
filtered_df.replace('&#8211;', '-', regex = True, inplace = True)
filtered_df.replace('&#8212;','-', regex = True, inplace = True)
filtered_df.replace('&#8208;','-', regex = True, inplace = True)
filtered_df.replace('&amp;','&', regex = True, inplace = True)
filtered_df.replace('&lt;', ' ' , regex = True, inplace = True)
filtered_df.replace('&#167;', '; ' , regex = True, inplace = True)
filtered_df.replace('br/&gt;', ', ' , regex = True, inplace = True)
filtered_df.replace('&#226;', ' ' , regex = True, inplace = True)
filtered_df.replace('&#191;', '' , regex = True, inplace = True)
filtered_df.replace('&#160;', ' ' , regex = True, inplace = True)

In [18]:
# Replace pd.NaT on datetime columns to 'None' because pd.NaT is not accepted in MongoDB.

filtered_df['PostDate'].replace({pd.NaT: None}, inplace = True)
filtered_df['CloseDate'].replace({pd.NaT: None}, inplace = True)
filtered_df['LastUpdatedDate'].replace({pd.NaT: None}, inplace = True)
filtered_df['ArchiveDate'].replace({pd.NaT: None}, inplace = True)

In [19]:
# Because the funding announcement that this column is referencing will not be found own our site, 
# we replaced the text "See Section III" with "please click on the link and go to related documents" to direct users back to grant.gov to see the attached funding announcements  

filtered_df['AdditionalInformationOnEligibility'] = filtered_df.AdditionalInformationOnEligibility.str.replace(r'(^.*See Section III.*$)', 'Please click on the link and go to related documents for more information.', regex=True)

In [20]:
# Many of the BLM grants listed are for specific states and are indicated in the grant's Opportunity Title
# To reduce the number of grants in our database that are not relevant to Michigan applicants, we filtered out several of the common states listed 

filtered_df = filtered_df[~filtered_df.OpportunityTitle.str.contains('|'.join(['AK', 'Alaska', 'AZ', 'Arizona','NM', 'New Mexico', 'HI', 'Hawaii', 'OR', 'Oregon', 'CO', 'Colorado', 'FL', 'Florida']))]

In [21]:
# Export data to CSV

#filtered_df.to_csv('filtered_grants.csv', index = False)

In [ ]:
# Create updated dictionary using the 'records’ format : list like [{column -> value}, … , {column -> value}]

updated_dict = filtered_df.to_dict('records')

### Uploading the updated/filtered grants.gov data into MongoDB

In [ ]:
# connect to MongoDB - change the << MONGODB URL >> to reflect your own connection string

client = pymongo.MongoClient("mongodb+srv://<<username>>:<<password>>@cluster0.mktistr.mongodb.net/?retryWrites=true&w=majority", server_api=ServerApi('1'))
db = client.admin

In [ ]:
# Check the server status:

serverStatusResult=db.command("serverStatus")
print(serverStatusResult)

In [ ]:
# Connect to the grantaccess database in MongoDB

#db = client.grantaccess
db = client.grantaccess

In [ ]:
# Connect to the "Grants" collection (a.k.a - table)

collection = db["Grants"]

#### Initial Push

In [ ]:
#Insert a list of dictionaries instead of a pandas df --> https://stackoverflow.com/questions/49221550/pandas-insert-a-dataframe-to-mongodb
# Slow process - should consider updating filter to specific grants instead of uploading all of it.

#for i in range(1,len(updated_dict)):
    #collection.insert_one(updated_dict[i])

#### Database Update

#### Seudo-code:
Call the data in MongoDB (previous version of the database) - comes as a list of dictionaries  
Compare to the new version of XML extract  
 
If 'OpportunityID' is not in database then add the new row of data to database  
If 'OpportunityID' is in database and have an updated 'LastUpdatedDate' then update row in database  
Else do nothing

new grants identified - collection.insert_one(); updated grants - collection.update_one()

In [ ]:
# Bring each document (row of data) stored in MongoDB into python as a dictionary dtype

old_data_list = []
for old_data in collection.find({}):
    old_data_list.append(old_data)

In [ ]:
# Creates a dataframe from the list of dictionaries

old_data_df = pd.DataFrame(old_data_list)

In [ ]:
old_data_df.info()

In [ ]:
# Identify new grants and select only the new grants
new_grants_df = filtered_df[~filtered_df['OpportunityID'].isin(old_data_df['OpportunityID'])]

# Create a list of dictionaries, each dictionary is one record
new_grants_dict = new_grants_df.to_dict('records')

# Upload each new record to MongoDB
for new_record in new_grants_dict:
    collection.insert_one(new_record)


In [ ]:
# Use updated_grants_dict to update the MongoDB database

updated_grants_df = filtered_df[filtered_df['LastUpdatedDate'].isin(old_data_df['LastUpdatedDate'])== False]

# Create a list of dictionaries, each dictionary is one record
updated_grants_dict = updated_grants_df.to_dict('records')

# Replace each updated record to MongoDB
for record in updated_grants_dict:
    record_to_update_dict = [match for match in old_data_list if match['OpportunityID'] == record['OpportunityID']][0]
    collection.update_one({'_id':record_to_update_dict['_id']}, {'$set': record}, upsert = False) 

References:

https://www.grants.gov/help/html/help/index.htm#t=Grantors%2FCreateGrantOpportunities.htm  
https://grantsgovprod.wordpress.com/2019/06/05/distinguishing-among-different-types-of-federal-awards-including-block-grants-cooperative-agreements-more/